# Report on Adversarial Examples
This notebook should 
1. Train new models with different parameters
1. for each model, generate an **untargeted** PGD attack
1. Plot the results

Read these notes before running stuff :) 


In [ ]:
%load_ext autoreload
%autoreload 2

import subprocess
from pathlib import Path 
from matplotlib import pyplot as plt 
import json 
import optuna 
from model_utils import find_best_model, choose_model
import numpy as np
import torch

prefix = 'experimentation/'
adv_dir = 'attacks/'

## Utilities for reporting accuracies

In [ ]:
def load_stats(dir):
    with open(dir) as f:
        stats = json.load(f)
    return stats 

def get_stats_regular(which=1, duration=100, model='max', root=prefix, dir=None):
    """ 
    Load stats of model specified directly using the directory of the model, or using which, duration and model name ('max' for latest)
    """
    if dir is None:
        dir = Path(root) / f'{which=}_{duration=}'
    model = choose_model(dir, model)
    dir = Path(model)
    return load_stats(dir / 'stats.json') 

def get_stats_attack(which=1, duration=100, model='max', root=prefix, adv_dir=adv_dir, dir=None, attacks=['targeted', 'targeted_increase', 'targeted_decrease', 'untargeted']):
    """ 
    Load adversarial stats of model specified directly using the directory of the model, 
    or using which, duration and model name ('max' for latest).
    Returns a dict of dicts, where the outer dict has attack types as keys and the inner uses perturbation sizes
    """
    if dir is None:
        # dir is not specified so we find it based on other information
        dir = Path(root) / f'{which=}_{duration=}'
        model = choose_model(dir, model)
        dir = model / adv_dir 
    # create a dict of dicts based on attack type and perturbation size
    retval = {}
    for p in dir.iterdir():
        key = str(p).split('/')[-1]
        perturbation_size = key.split('=')[-1]
        for attack in attacks:
            if f"attack_type='{attack}'" in key: 
                if not attack in retval.keys():
                    retval[attack] = {}
                try:
                    retval[attack][perturbation_size] = load_stats(p / 'adv_stats.json')
                except FileNotFoundError:
                    retval[attack][perturbation_size] = {'acc': np.nan, 'cm': np.zeros((3, 3))}

    return retval

## Train New Models
Each with a specific set of parameters, including data parameters

In [ ]:
def make_args(param):
    """
    turn parameter dictionary into list of arguments
    """
    out = []
    for k, v in param.items():
        out.append('--' + k)
        out.append(str(v))
    return out 

durations = [10, 20, 50]
whichs = [1, 3]
n_epochs = 30
hs = 64 
dropout = 1/3
num_layers = 2
thresholds = [.1, .15, .25, .35]
n_classes = len(thresholds) + 1
opt_time_seconds = 10*60
model_params = {
    f'{prefix}{which=}_{duration=}': {
        'duration': duration, 'which': which, 'n_classes': n_classes, 'n_epochs': n_epochs,
        'dropout': dropout, 'hs': hs, 'num_layers': num_layers, 'thresholds': thresholds
    } 
    for duration in durations for which in whichs
}



### ATTENTION
The code below takes time to run and should only be run if you want to make your own model. It is not necessary for the majority of this notebook, since the results are already on github. 

In [ ]:
# create models
for name, v in model_params.items():
    which = v['which']
    duration = v['duration']
    def objective(trial):
        v['dropout'] = trial.suggest_float('dropout', 0, .5)
        v['num_layers'] = trial.suggest_int('num_layers', 1, 4)
        v['hs'] = trial.suggest_int('hs', 64, 256, log=True)
        print('\n\n', v, '\n\n')
        subprocess.run(['python', 'lstm.py'] + make_args(v) + ['--target_directory', name]) 
        acc = get_stats_regular(which=which, duration=duration)['acc']
        return -acc # maximize acc

    study = optuna.create_study()
    study.enqueue_trial({'dropout': dropout, 'num_layers': num_layers, 'hs': hs})
    study.optimize(objective, timeout=opt_time_seconds)

## Save (sequential) model predictions on regular test data
### ATTENTION:
Again, you don't have to run this if you downloaded the results already

In [ ]:
for name, v in model_params.items():
    which = v['which']
    duration = v['duration']
    model_path = choose_model(Path(f'{prefix}/{which=}_{duration=}'), 'best') 
    subprocess.call(['python', 'test_sequential.py', '--model_path', model_path])

In [ ]:
# # find the best accuracy in the directories
# best_accs = {}
# for name, v in model_params.items():
#     best_accs[name] = find_best_model(name)
# print(*[v[1] for v in best_accs.values()], sep='\n')


## Generate Adversarial Examples
Save the results with the model in the appropriate folder, both for targeted and untargeted attacks. 

Parameters of the attack:
- Attack type
- Perturbation size

In [ ]:
# For the case of breaking the process, it is nice to know the order of the scripts executed
# Generate that order
order = []
attack_types = ['untargeted'] # ['targeted_increase', 'targeted_decrease', 'targeted', 'untargeted']
perturbation_sizes = [.1*i for i in range(6)]
for name in model_params.keys():
    attack_params = {
        f'{adv_dir}{attack_type=}_{perturbation_size=}/': {'attack_type': attack_type, 'perturbation_size': perturbation_size} 
        for attack_type in attack_types for perturbation_size in perturbation_sizes
    }
    order.extend([(name, attack_name) for attack_name in attack_params.keys()])

def later_experiment(comp, ref, order=order):
    """
    Check if comp is later than ref in order
    """
    if ref == None:
        return True
    if ref =='skip_all':
        return False 
    j = len(order)
    for i, o in enumerate(order):
        if str(o[0]) == str(comp[0]) and str(o[1]) == str(comp[1]):
            j = i 
            break 
    for o in order[:j+1]:
        if str(o[0]) == str(ref[0]) and str(o[1]) == str(ref[1]):
            return True 
    return False 

### ATTENTION:
Again, you don't have to run this if you downloaded the results already

In [ ]:
# for each model, make a number of attacks 
# To that end, make a subfolder "attacks" for each of the models

# ref = (f'{prefix}/which=4_duration=150', "attacks/attack_type='targeted'_perturbation_size=0.01/") 
ref = None
# ref = 'skip_all'

for name in model_params.keys():
    attack_params = {
        f'{adv_dir}{attack_type=}_{perturbation_size=}/': {'attack_type': attack_type, 'perturbation_size': perturbation_size} 
        for attack_type in attack_types for perturbation_size in perturbation_sizes
    }
    for attack_name, v in attack_params.items():
        if later_experiment((name, attack_name), ref):
            print('doing', (name, attack_name))
            if 'increase' in name:
                preference = 'increase'
            elif 'decrease' in name:
                preference = 'decrease'
            else:
                preference = 'None'
            subprocess.run(['python', 'attack.py'] + make_args(v) + ['--source_directory', name, '--target_directory', attack_name, '--model_name', 'best', '--preference', preference, '--validation_data', 'False'])
        else:
            print('skipped', (name, attack_name))

## Plot the Results

In [ ]:
# class_to_duration_mapping = {0: 0., 1: .125, 2: .2, 3: .3, 4: .7}

def pred_to_cycles(y, idx, data_metadata):
    thresholds = data_metadata['thresholds']
    if idx >= len(y): return np.nan # the model did not recommend to switch
    predicted_class = int(y[idx])
    max_life = thresholds[-1] * 3
    if predicted_class == n_classes - 1:
        if int(y[-1]) == n_classes - 1:
            return ((len(y) - idx - 1) * 10 + thresholds[-1]) / max_life
            # return np.nan # cannot say what the actual remaining life is
        else:
            for i, v in enumerate(y):
                if v < n_classes - 1:
                    break 
            return ((i - idx) * 10 + (thresholds[-2] + thresholds[-1]) / 2) / max_life 
    if predicted_class == 0: return 0. 
    return (thresholds[predicted_class] + thresholds[predicted_class - 1]) /2 / max_life

def cost_of_matrix(cm, critical_class, c_over=15.1, c_under=40):
    """ 
    Compute the cost according to the relevant confusion matrix
    """
    cm = np.array(cm)
    # find the number of cases where we predict a non-critical class when it is in fact critical
    n_underestimate = np.sum(cm[critical_class+1:, :critical_class]) # predicted class is critical_class + 1 or higher
    # find the number of non-critical cases which are predicted as critical
    n_overestimate = np.sum(cm[:critical_class, critical_class+1:])
    return (n_underestimate * c_under + n_overestimate * c_over) / np.sum(cm)

def num_cycles_left(preds, ys, rule, data_metadata):
    stop = rule(preds) # last sample to include 
    return pred_to_cycles(ys, stop, data_metadata)

def load_p_y(model_path, errors=None):
    with open(Path(model_path) / 'preds_ys.json') as f:
        preds_ys = json.load(f)
    preds = preds_ys['preds']
    ys = preds_ys['ys']
    if errors is not None:
        for engine, t, error in errors:
            preds[engine][t] = error
    return preds, ys 

def mean_num_cycles_left(model_path, rule, errors, data_metadata):
    preds, ys = load_p_y(model_path, errors)
    res = np.zeros((len(preds), ))
    for i, (p, y) in enumerate(zip(preds, ys)):
        res[i] = num_cycles_left(p, y, rule, data_metadata)
    return np.nanmean(res)

def too_early(stop, ys, threshold):
    # whether the engine is taken out at a stage earlier than it should have been
    if stop+1 >= len(ys): return False 
    return ys[stop + 1] >= threshold # next sample could also have been included

def too_late(stop, ys, threshold=0):
    if stop >= len(ys):
        return ys[-1] <= threshold
    return ys[stop] <= threshold

def too_early_rate(model_path, rule, threshold, errors=None):
    preds, ys = load_p_y(model_path, errors) 
    num_early = 0
    for p, y in zip(preds, ys):
        stop = rule(p)
        num_early += int(too_early(stop, y, threshold))
    return num_early / len(preds)

def too_late_rate(model_path, rule, threshold=0, errors=None):
    preds, ys = load_p_y(model_path, errors) 
    num_late = 0
    for p, y in zip(preds, ys):
        stop = rule(p)
        num_late += int(too_late(stop, y, threshold))
    return num_late / len(preds)

def load_errors(attack):
    """ 
    take a folder "attack", containing examples, and return a list of (engine, time_idx, adv_lbl)
    """
    examples = Path(attack) / 'examples'
    ids_times = torch.load(examples /'ids_times.pt')
    adv_lbl = torch.load(examples / 'adv_lbl.pt')
    return [(i[0], i[1], l) for i, l in zip(ids_times, adv_lbl)]

def cost_of_model(model_path, rule='matrix', thresholds=[0, 4], C1=32.5, C2=4, C3=11.1, mean_rul=.5, cm=None, errors=None):
    """
    Find the cost of using one model depending on some decision rule and the model's performance
    """
    if rule == 'matrix': 
        if cm is None:
            with open(model_path / 'stats.json') as f:
                stats = json.load(f)
            try: cm = stats['cm']
            except KeyError: cm = stats['confusion_matrix']
        return cost_of_matrix(cm, thresholds[1])
    else:
        with open(model_path / 'data_metadata.json') as f:
            data_metadata = json.load(f)
        cycles_left = mean_num_cycles_left(model_path, rule, errors, data_metadata)
        # print(cycles_left)
        p1 = too_early_rate(model_path, rule, thresholds[-1], errors) # underestimation of remaining life
        p2 = too_late_rate(model_path, rule, thresholds[0], errors) # overestimation of remaining life
        # print(f'too early {p1=}, too late {p2=}')
        cost = cycles_left * (C2 + C3/14) + p1 * (C2 + C3/14) + p2 * (C1 + C3) 
        return cost # / c_baseline

def cost_of_sequence_attacks(model_path, rule, thresholds=[0, 3], C1=32.5, C2=4, C3=11.1, mean_rul=.5, attacks=['untargeted'], perturbation_sizes=[0.]):
    """
    For a given model and attack parameters, find the cost in the different attack cases
    """
    results = {a: [] for a in attacks}
    for attack in attacks:
        for eps in perturbation_sizes:
            if eps > 1e-4:
                errors = load_errors(model_path / adv_dir / f'attack_type=\'{attack}\'_perturbation_size={eps}')
            else:
                errors = None 
            cost = cost_of_model(model_path, thresholds=thresholds, C1=C1, C2=C2, C3=C3, mean_rul=mean_rul, errors=errors, rule=rule)
            results[attack].append(cost)
    return results

In [ ]:

C1=32.5
C2=4
C3=11.1

baseline_rates = [.15, .2, .25, .3, .35]
baseline_rates = [r*(C2 + C3/14) for r in baseline_rates]
thresholds = [0, 3]

def plot_sequence_cost(which, duration, rule, attack='untargeted', perturbation_sizes=perturbation_sizes):
    path = Path(f'{prefix}/{which=}_{duration=}')
    model = choose_model(path, model='best')
    # print(which, duration)
    results = cost_of_sequence_attacks(model, rule, attacks=[attack], thresholds=thresholds, perturbation_sizes=perturbation_sizes, C1=C1, C2=C2, C3=C3)[attack]
    return results

def rule(preds):
    """
    find the last time point in preds which we include
    """
    for i, p in enumerate(preds):
        if p < 2 or (i > 1 and p < 3 and preds[i-1] < 3):
            return i 
    return len(preds)

fig, axs = plt.subplots(1, len(whichs), figsize=(10, 5))
for i, which in enumerate(whichs):
    ax = axs[i]
    for duration in durations:
        res = plot_sequence_cost(which, duration, rule)
        ax.plot(perturbation_sizes, res, label=f'Duration = {duration}')
    ax.legend()
    ax.set_title(f'Cost using dataset {which}')
    ax.hlines(baseline_rates, min(perturbation_sizes), max(perturbation_sizes), linestyle='--') # color=['lightcoral', 'indianred', 'brown'], 
plt.show()

In [ ]:

def plot_statistics(stat='acc', critical_class=3, attacks=['targeted', 'untargeted', 'targeted_increase', 'targeted_decrease']):
    def plot_accs(which, attack, filename=None):
        fig, ax = plt.subplots(1, 1)
        ax.set_title(f'{attack}_{which=}')
        for duration in durations:
            key = f'{which=}_{duration=}'
            if stat == 'acc':
                y = [adv_accs[key][attack][str(s)]['acc'] for s in perturbation_sizes]
            elif stat == 'cost':
                y =[
                    cost_of_matrix(adv_accs[key][attack][str(s)]['cm'], critical_class) for s in perturbation_sizes
                ]
            ax.plot(x, y, label=key)
            # ax.set_ylim(.5, 1)
            ax.grid()
            ax.set_xlabel('perturbation size')
            ax.set_ylabel(stat)
        ax.legend()
        if filename is not None: 
            dir = Path('/'.join(filename.split('/')[:-1]))
            dir.mkdir(exist_ok=True, parents=True)
            fig.savefig(filename)
        return fig, ax 
    # get accs
    regular_accs = {}
    adv_accs = {}
    for which in whichs:
        for duration in durations:
            key = f'{which=}_{duration=}'
            regular_accs[key] = get_stats_regular(which, duration, model='best')
            adv_accs[key] = get_stats_attack(which, duration, model='best')

    # plot targeted results, resulting in a plot per "which"
    # x=perturbation_size, y=acc
    # where we have one line for each duration
    x = perturbation_sizes 
    for which in whichs:
        for attack in attacks:
            fig, ax = plot_accs(which, attack, filename=f'plots/{prefix}/{which=}_{duration=}_{attack=}_{stat=}.png')

        # plt.show()

In [ ]:
plot_statistics(stat='acc', attacks=attack_types)

In [ ]:
plot_statistics(stat='cost', critical_class=2, attacks=attack_types) # old way of computing cost

## Get a more visual idea of what these attacks are doing

In the plots below, a star marks that we could perturb a model to predict this value. The regular predictions are crosses and the true values are circles

In [ ]:
# check out where we could introduce errors
attack = 'untargeted'
perturbation_size = perturbation_sizes[-2]
which = 1 
duration = 50
chosen_model = choose_model(f'{prefix}/{which=}_{duration=}', 'best')
examples = chosen_model / f'{adv_dir}/attack_type=\'{attack}\'_{perturbation_size=}/examples/'
attack_ids = torch.load(examples / 'ids.pt')
engines_times = torch.load(examples/'ids_times.pt')
adv_lbl = torch.load(examples / 'adv_lbl.pt')
engine_errors = {v.item(): [] for v in torch.unique(engines_times[:, 0])}
for i, (e, t) in enumerate(engines_times):
    engine_errors[e.item()].append((t.item(), adv_lbl[i].item()))
print(engine_errors)

In [ ]:
with open(chosen_model /'preds_ys.json') as f:
    preds_ys = json.load(f)

for engine, vals in engine_errors.items():
    time = [v[0] for v in vals]
    cl = [v[1] for v in vals]
    preds = preds_ys['preds'][engine]
    ys = preds_ys['ys'][engine]
    plt.plot(np.arange(len(preds)), ys, 'o')
    plt.plot(np.arange(len(preds)), preds, 'x')
    plt.plot(time, cl, '*')
    plt.ylim((-.5, n_classes))
    plt.show()